In [48]:
# # Use the latest version of pip.
# !pip install --upgrade pip
# !pip install --upgrade "tfx[kfp]"
# !pip install --upgrade tensorflow
# !pip install --upgrade tensorflow_transform

# !pip install --upgrade pyarrow
# !pip install --upgrade apache-beam
# !pip install --upgrade tensorflow_model_analysis


In [49]:
# !pip install apache-beam pyarrow tensorflow_model_analysis tfx

In [50]:
# !pip install tensorflow-model-analysis==0.46.0

In [51]:
# !dir

In [52]:
# !pip install -r final_r.txt

In [53]:
# import tensorflow_model_analysis as tfma
# from tfx import v1 as tfx

# !pip uninstall pandas -y
# !pip install pandas

In [1]:
import os
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils
from tensorflow.keras import layers
import tensorflow_model_analysis as tfma
from tfx import v1 as tfx
from tfx.types import Channel, standard_artifacts
from tfx.dsl.components.common.resolver import Resolver
from tfx.components import Evaluator
from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tfx.orchestration.pipeline import Pipeline
from tfx.proto import trainer_pb2, pusher_pb2
from tfx.types import standard_artifacts
# import tensorflow_model_analysis as tfma
from google.cloud import bigquery
import pandas as pd
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
from google.cloud import storage

2024-07-21 18:03:03.954500: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-21 18:03:04.840125: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-21 18:03:04.840292: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-21 18:03:05.026063: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-21 18:03:05.408593: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-21 18:03:05.412788: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

### Check the package versions.

In [3]:

import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import kfp
print('KFP version: {}'.format(kfp.__version__))
print('TFMA version: {}'.format(tfma.__version__))

TensorFlow version: 2.15.1
TFX version: 1.15.1
KFP version: 1.8.22
TFMA version: 0.46.0


In [4]:
GOOGLE_CLOUD_PROJECT = 'jovial-sight-429218-i7'
GOOGLE_CLOUD_REGION = 'us-central1'
GCS_BUCKET_NAME = 'mlops_final_run'
FILE_NAME ='Final_Chicago_Train.csv'

#### Set `gcloud` to use your project.

In [5]:
 {GOOGLE_CLOUD_PROJECT}

{'jovial-sight-429218-i7'}

### Set up Global variables for model serving locations

In [6]:


# Initialize the Google Cloud Storage client
client = storage.Client()


PIPELINE_NAME = 'chicago-gcp-vertex-pipelines'

# Function to create directories in GCS
def create_gcs_directory(bucket_name, path):
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(path)
    blob.upload_from_string('', content_type='application/x-www-form-urlencoded;charset=UTF-8')

# Paths for pipeline artifacts
PIPELINE_ROOT = f'gs://{GCS_BUCKET_NAME}/{PIPELINE_NAME}/pipeline_root/'
MODULE_ROOT = f'gs://{GCS_BUCKET_NAME}/{PIPELINE_NAME}/pipeline_module/'
DATA_ROOT = f'gs://{GCS_BUCKET_NAME}/{PIPELINE_NAME}/data/'
SERVING_MODEL_DIR = f'gs://{GCS_BUCKET_NAME}/{PIPELINE_NAME}/serving_model/'

# Create directories
create_gcs_directory(GCS_BUCKET_NAME, f'{PIPELINE_NAME}/pipeline_root/')
create_gcs_directory(GCS_BUCKET_NAME, f'{PIPELINE_NAME}/pipeline_module/')
create_gcs_directory(GCS_BUCKET_NAME, f'{PIPELINE_NAME}/data/')
create_gcs_directory(GCS_BUCKET_NAME, f'{PIPELINE_NAME}/serving_model/')

print('GCS directories created:')
print(PIPELINE_ROOT)
print(MODULE_ROOT)
print(DATA_ROOT)
print(SERVING_MODEL_DIR)


GCS directories created:
gs://mlops_final_run/chicago-gcp-vertex-pipelines/pipeline_root/
gs://mlops_final_run/chicago-gcp-vertex-pipelines/pipeline_module/
gs://mlops_final_run/chicago-gcp-vertex-pipelines/data/
gs://mlops_final_run/chicago-gcp-vertex-pipelines/serving_model/


We need to make our own copy of the dataset. Because TFX ExampleGen reads
inputs from a directory, we need to create a directory and copy dataset to it
on GCS.

Take a quick look at the CSV file.

In [7]:


client = bigquery.Client()
# Define the SQL query
query = """
SELECT * FROM bigquery-public-data.chicago_taxi_trips.taxi_trips
WHERE EXTRACT(year FROM trip_start_timestamp) > 2018 AND
      trip_miles IS NOT NULL AND
      trip_seconds IS NOT NULL
ORDER BY RAND()
LIMIT 40000
"""

# Run the query
query_job = client.query(query)

# Convert to DataFrame
df = query_job.to_dataframe()

# Display the DataFrame
df




,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,a8f3ac8c8b7751248924252b6880ce393a3b486f,9f2cb5811cb0450b60400085b275296a3a307c2c26458f...,2019-05-09 13:00:00+00:00,2019-05-09 13:15:00+00:00,426,0.74,17031081402,17031081401,8,8,...,0.0,6.00,Cash,Flash Cab,41.891972,-87.612945,POINT (-87.6129454143 41.8919715078),41.895033,-87.619711,POINT (-87.6197106717 41.8950334495)
1,74e380607a4462d41cca62cd6beec24ed74357fd,501280246f63bb05a144c72919f7c442c3a50fd9cfc2c7...,2020-02-20 18:15:00+00:00,2020-02-20 18:30:00+00:00,585,1.57,17031081700,17031080201,8,8,...,0.0,11.25,Credit Card,Flash Cab,41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),41.909496,-87.630964,POINT (-87.630963601 41.9094956686)
2,69162dda4acf3f1a1616a709a6d92f4bfb689818,c26296b24f8bf5e5dd0751206051b0bac76a39d87ec36d...,2019-05-31 11:45:00+00:00,2019-05-31 11:45:00+00:00,315,0.61,17031839100,17031839100,32,32,...,0.0,5.50,Cash,Chicago Carriage Cab Corp,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707)
3,5c519eb8e532ffbc9f7c5c0bd0b2e1f5b76176f2,1833edab9ed696386a8224986104fd046eaf9ea8dffb9e...,2019-01-10 02:15:00+00:00,2019-01-10 02:15:00+00:00,420,0.10,<NA>,<NA>,8,24,...,0.0,10.25,Credit Card,Taxi Affiliation Services,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.901207,-87.676356,POINT (-87.6763559892 41.90120699410001)
4,7a8120b81ee6923fa0fc017fb61173bd307cea95,bf2d7795769ed240df920dce5605bda9e543e9cf05272a...,2021-12-17 16:15:00+00:00,2021-12-17 19:00:00+00:00,9720,104.30,<NA>,<NA>,<NA>,<NA>,...,25.0,269.75,Cash,"Taxicab Insurance Agency, LLC",NaN,NaN,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,1b977c00fa841ed8c542b9a1b4ad088293f6c7c6,34a67a4a44a0c07c24568bbdee112b8e7877b6b49664a7...,2021-10-14 17:30:00+00:00,2021-10-14 18:30:00+00:00,3918,18.04,17031081500,17031980000,8,76,...,1.0,48.75,Cash,Flash Cab,41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),41.979071,-87.903040,POINT (-87.9030396611 41.9790708201)
39996,b935a4a925089bf0ceb2036fdd87c17ddef89651,c0233d1acdf95f778faf4bcadcff7035dc6bbcab99f07e...,2023-07-13 12:45:00+00:00,2023-07-13 12:45:00+00:00,480,1.50,17031081201,17031081800,8,8,...,0.0,7.50,Cash,U Taxicab,41.899156,-87.626211,POINT (-87.6262105324 41.8991556134),41.893216,-87.637844,POINT (-87.6378442095 41.8932163595)
39997,2619ebf9c87ce668bd78e9b816d5ac3e5831af66,e931cc8f41df1e511815b487341e4896fb01f01b2f8fc6...,2019-07-26 10:45:00+00:00,2019-07-26 10:45:00+00:00,480,1.20,17031839100,17031081403,32,8,...,0.0,9.00,Credit Card,Choice Taxi Association,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.890922,-87.618868,POINT (-87.6188683546 41.8909220259)
39998,14f5937595f891ff505aa906a418f3d61b8277c2,dd3ec76231a679ee3501f302592955447df6907cf2806e...,2019-07-18 13:00:00+00:00,2019-07-18 13:15:00+00:00,660,0.90,17031320100,17031081800,32,8,...,0.0,7.25,Cash,Top Cab Affiliation,41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),41.893216,-87.637844,POINT (-87.6378442095 41.8932163595)


In [8]:
df.isnull().sum()

unique_key                    0
taxi_id                       0
trip_start_timestamp          0
trip_end_timestamp            0
trip_seconds                  0
trip_miles                    0
pickup_census_tract       19660
dropoff_census_tract      19910
pickup_community_area      2727
dropoff_community_area     4131
fare                         22
tips                         22
tolls                       294
extras                       22
trip_total                   22
payment_type                  0
company                       0
pickup_latitude            2721
pickup_longitude           2721
pickup_location            2721
dropoff_latitude           3964
dropoff_longitude          3964
dropoff_location           3964
dtype: int64

In [9]:
df["trip_start_day"] = df.trip_start_timestamp.apply(lambda x: x.day)
df["trip_end_day"] = df.trip_end_timestamp.apply(lambda x: x.day)

In [10]:
df["trip_start_month"] = df.trip_start_timestamp.apply(lambda x: x.month)
df["trip_end_month"] = df.trip_end_timestamp.apply(lambda x: x.month)


In [11]:
df["trip_start_hour"] = df.trip_start_timestamp.apply(lambda x: x.hour)
df["trip_end_hour"] = df.trip_end_timestamp.apply(lambda x: x.hour)

In [65]:
df

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location,trip_start_day,trip_start_month,trip_start_hour
0,4b0ff5638ed5905d99da624abc903b967f868cfd,3426c4a607f9804cc5f33ecb1cab61f70f63b3f0593c94...,2019-03-11 12:45:00+00:00,2019-03-11 12:45:00+00:00,340,0.84,17031839100,17031280100,32,28,...,24 Seven Taxi,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.885300,-87.642808,POINT (-87.6428084655 41.8853000224),11,3,12
1,eb868659d19997db5a97e8ae22097ce12fa3c8e0,5776c9e3fe3235f1c036220f324b07aa8728ebd6641bfc...,2019-03-11 22:45:00+00:00,2019-03-11 23:15:00+00:00,1213,7.66,<NA>,<NA>,32,6,...,Chicago Carriage Cab Corp,41.878866,-87.625192,POINT (-87.6251921424 41.8788655841),41.944227,-87.655998,POINT (-87.6559981815 41.9442266014),11,3,22
2,3cd47551b2424db69e6923b7ae4929fd7f04eed2,40dd26181941ac03ca95b3d4cf2f0d12b34ae0338e688b...,2023-05-12 07:00:00+00:00,2023-05-12 07:15:00+00:00,1134,5.47,<NA>,<NA>,23,28,...,Flash Cab,41.900070,-87.720918,POINT (-87.7209182385 41.9000696026),41.874005,-87.663518,POINT (-87.6635175498 41.874005383),12,5,7
3,5ded2218e2090f75607f2b03d00265ef69d9d71f,1d72db3a18692cc5b5e4ea41bb1de0e45d4149b495dbd0...,2021-06-13 12:45:00+00:00,2021-06-13 13:00:00+00:00,563,1.62,<NA>,<NA>,3,77,...,Flash Cab,41.965812,-87.655879,POINT (-87.6558787862 41.96581197),41.986712,-87.663416,POINT (-87.6634164054 41.9867117999),13,6,12
4,67fd7d0b6a607a3d228738481960eb1dd14079ad,48c3c22d766613be3982924a72efdc03758b9666b3996b...,2023-01-09 10:30:00+00:00,2023-01-09 10:45:00+00:00,420,1.40,<NA>,<NA>,28,28,...,"Taxicab Insurance Agency, LLC",41.874005,-87.663518,POINT (-87.6635175498 41.874005383),41.874005,-87.663518,POINT (-87.6635175498 41.874005383),9,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,fe24c1fa80ac672a7c691ec4e372349dedb12c82,1aa27355c709dc0cba3790a2549861c6b7864165ccf020...,2022-11-02 18:00:00+00:00,2022-11-02 18:00:00+00:00,0,0.00,<NA>,<NA>,75,75,...,Taxi Affiliation Services,41.689730,-87.669054,POINT (-87.6690544032 41.6897299145),41.689730,-87.669054,POINT (-87.6690544032 41.6897299145),2,11,18
39996,149b3c85da0c94caa99aab8a3d6baf5a0299aec3,91996675edbfbd4ed70014061e791e1f42a90050307183...,2019-08-16 16:00:00+00:00,2019-08-16 16:30:00+00:00,1560,0.10,17031071500,17031839100,7,32,...,Taxi Affiliation Services,41.914616,-87.631717,POINT (-87.6317173661 41.9146162864),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),16,8,16
39997,f680d345c22ff8a161db7f8349f7b609d650b634,878fad758bcb6e1a1444c81a48594c00962e442e20bb71...,2020-02-18 18:15:00+00:00,2020-02-18 18:30:00+00:00,861,1.58,17031081800,17031081300,8,8,...,Chicago Carriage Cab Corp,41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),41.898332,-87.620763,POINT (-87.6207628651 41.8983317935),18,2,18
39998,c180c8dd0502925daf32f79221cc68c92a287b77,d899402f96aeec3adabbee95f3a99aba9ade0e0b5c1d93...,2019-03-05 09:15:00+00:00,2019-03-05 09:30:00+00:00,1087,2.90,17031080100,17031281900,8,28,...,City Service,41.907520,-87.626659,POINT (-87.6266589003 41.90752007470001),41.879255,-87.642649,POINT (-87.642648998 41.8792550844),5,3,9


In [12]:
df.to_csv("Final_Chicago_Train.csv", index=False)

In [13]:
!gsutil mb -l {GOOGLE_CLOUD_REGION} gs://{GCS_BUCKET_NAME}
!gsutil cp Final_Chicago_Train.csv {DATA_ROOT}/

Creating gs://mlops_final_run/...
ServiceException: 409 A Cloud Storage bucket named 'mlops_final_run' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.
Copying file://Final_Chicago_Train.csv [Content-Type=text/csv]...
- [1 files][ 16.4 MiB/ 16.4 MiB]                                                
Operation completed over 1 objects/16.4 MiB.                                     


# Write Example Component


### Write model code.

We will use the same model code as in the
[Simple TFX Pipeline Tutorial](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple).

In [14]:
_trainer_module_file = 'bfs_trainer.py'
_transformer_module_file = 'transformer.py'
_training_pipeline_file = 'training_pipeline.py'

In [15]:
%%writefile {_transformer_module_file}

import tensorflow as tf
import tensorflow_transform as tft

# Define the numerical features that will be used in the model.
NUMERICAL_FEATURES = ['trip_miles', 'trip_seconds']

# Define the features that will be bucketized.
BUCKET_FEATURES = [
    'pickup_latitude', 'pickup_longitude', 'dropoff_latitude',
    'dropoff_longitude'
]

# Define the number of buckets used by tf.transform for encoding each feature in BUCKET_FEATURES.
FEATURE_BUCKET_COUNT = 10

# Define the categorical features that are represented as numerical values.
CATEGORICAL_NUMERICAL_FEATURES = [
    'trip_start_hour', 'trip_start_day', 'trip_start_month',
    'pickup_census_tract', 'dropoff_census_tract', 'pickup_community_area',
    'dropoff_community_area'
]

# Define the categorical features that are represented as strings.
CATEGORICAL_STRING_FEATURES = [
    'payment_type',
    'company',
]

# Define the number of vocabulary terms used for encoding categorical features.
VOCAB_SIZE = 1000

# Define the count of out-of-vocab buckets in which unrecognized categorical are hashed.
OOV_SIZE = 10

# Define the keys for the label and fare columns in the input data.
LABEL_KEY = 'fare'

# Define a helper function that appends the suffix '_xf' to a feature key to avoid clashes
# with raw feature keys when running the Evaluator component.
def t_name(key):
    return key + '_xf'

def _make_one_hot(x, key):
    """Make a one-hot tensor to encode categorical features.
    Args:
        x: A dense tensor
        key: A string key for the feature in the input
    Returns:
        A dense one-hot tensor as a float list
    """
    # Computing and applying vocabulary to the input tensor and integerizing it.
    integerized = tft.compute_and_apply_vocabulary(x,
                                                   top_k=VOCAB_SIZE,
                                                   num_oov_buckets=OOV_SIZE,
                                                   vocab_filename=key,
                                                   name=key)
    # Getting the vocabulary size for the feature.
    depth = (
        tft.experimental.get_vocabulary_size_by_name(key) + OOV_SIZE)
    # Converting the integerized tensor to a one-hot tensor.
    one_hot_encoded = tf.one_hot(
        integerized,
        depth=tf.cast(depth, tf.int32),
        on_value=1.0,
        off_value=0.0)
    return tf.reshape(one_hot_encoded, [-1, depth])

def _fill_in_missing(x):
    """Replace missing values in a SparseTensor.
    Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
    Args:
      x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
        in the second dimension.
    Returns:
      A rank 1 tensor where missing values of `x` have been filled in.
    """
    if not isinstance(x, tf.sparse.SparseTensor):
        return x

    default_value = '' if x.dtype == tf.string else 0
    return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)

def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
      inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
      Map from string feature key to transformed feature operations.
    """
    outputs = {}
    for key in NUMERICAL_FEATURES:
        # Filling in missing values and scaling the numerical features to have mean=0 and variance=1.
        outputs[t_name(key)] = tft.scale_to_z_score(
            _fill_in_missing(inputs[key]), name=key)

    for key in BUCKET_FEATURES:
        # Filling in missing values and bucketizing the features.
        outputs[t_name(key)] = tf.cast(tft.bucketize(
            _fill_in_missing(inputs[key]), FEATURE_BUCKET_COUNT, name=key),
            dtype=tf.float32)

    for key in CATEGORICAL_STRING_FEATURES:
        # Filling in missing values and one-hot encoding the categorical string features.
        outputs[t_name(key)] = _make_one_hot(_fill_in_missing(inputs[key]), key)

    for key in CATEGORICAL_NUMERICAL_FEATURES:
        # Filling in missing values, converting the categorical numerical features to strings, and one-hot encoding them.
        outputs[t_name(key)] = _make_one_hot(tf.strings.strip(
        tf.strings.as_string(_fill_in_missing(inputs[key]))), key)

    # Fare is used as a label here
    outputs[LABEL_KEY] = _fill_in_missing(inputs[LABEL_KEY])

    return outputs



Overwriting transformer.py


In [16]:
%%writefile {_trainer_module_file}
# Import necessary libraries
from typing import Dict, List, Text
import os
import glob
from absl import logging
import datetime
import tensorflow as tf
import tensorflow_transform as tft
from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_transform import TFTransformOutput


_LABEL_KEY = 'fare'

_BATCH_SIZE = 40

def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
    """Generates features and label for tuning/training.

    Args:
      file_pattern: List of paths or patterns of input tfrecord files.
      data_accessor: DataAccessor for converting input to RecordBatch.
      tf_transform_output: A TFTransformOutput.
      batch_size: representing the number of consecutive elements of returned
        dataset to combine in a single batch

    Returns:
      A dataset that contains (features, indices) tuple where features is a
        dictionary of Tensors, and indices is a single Tensor of label indices.
    """
    # Create a dataset from the input files using the TFTransformOutput and batch size
    return data_accessor.tf_dataset_factory(
        file_pattern,
        tfxio.TensorFlowDatasetOptions(
            batch_size=batch_size, label_key=_LABEL_KEY),
        tf_transform_output.transformed_metadata.schema)

def _get_tf_examples_serving_signature(model, tf_transform_output):
    """Returns a serving signature that accepts `tensorflow.Example`."""

    # We need to track the layers in the model in order to save it.
    # TODO(b/162357359): Revise once the bug is resolved.
    model.tft_layer_inference = tf_transform_output.transform_features_layer()

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
    ])
    def serve_tf_examples_fn(serialized_tf_example):
        """Returns the output to be used in the serving signature."""
        raw_feature_spec = tf_transform_output.raw_feature_spec()
        # Remove label feature since these will not be present at serving time.
        raw_feature_spec.pop(_LABEL_KEY)
        raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
        transformed_features = model.tft_layer_inference(raw_features)
        logging.info('serve_transformed_features = %s', transformed_features)

        outputs = model(transformed_features)
        # TODO(b/154085620): Convert the predicted labels from the model using a
        # reverse-lookup (opposite of transform.py).
        return {'outputs': outputs}

    # Define a serving function that takes in serialized tf.Example and returns model outputs
    return serve_tf_examples_fn

def _get_transform_features_signature(model, tf_transform_output):
    """Returns a serving signature that applies tf.Transform to features."""

    # We need to track the layers in the model in order to save it.
    # TODO(b/162357359): Revise once the bug is resolved.
    model.tft_layer_eval = tf_transform_output.transform_features_layer()

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
    ])
    def transform_features_fn(serialized_tf_example):
        """Returns the transformed_features to be fed as input to evaluator."""
        raw_feature_spec = tf_transform_output.raw_feature_spec()
        raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
        transformed_features = model.tft_layer_eval(raw_features)
        logging.info('eval_transformed_features = %s', transformed_features)
        return transformed_features

    # Define a serving function that takes in serialized tf.Example and returns transformed features
    return transform_features_fn

def export_serving_model(tf_transform_output, model, output_dir):
    """Exports a keras model for serving.

    Args:
      tf_transform_output: Wrapper around output of tf.Transform.
      model: A keras model to export for serving.
      output_dir: A directory where the model will be exported to.
    """
    # Save the transform layer to the model for serving
    model.tft_layer = tf_transform_output.transform_features_layer()

    signatures = {
        'serving_default':
            _get_tf_examples_serving_signature(model, tf_transform_output),
        'transform_features':
            _get_transform_features_signature(model, tf_transform_output),
    }

    # Save the model with serving signatures
    model.save(output_dir, save_format='tf', signatures=signatures)

def _build_keras_model(tf_transform_output: TFTransformOutput
                       ) -> tf.keras.Model:
    """Creates a DNN Keras model for classifying taxi data.

    Args:
      tf_transform_output: [TFTransformOutput], the outputs from Transform

    Returns:
      A keras Model.
    """
    # Create a dictionary of model inputs based on the transformed feature spec
    feature_spec = tf_transform_output.transformed_feature_spec().copy()
    feature_spec.pop(_LABEL_KEY)

    inputs = {}
    for key, spec in feature_spec.items():
        if isinstance(spec, tf.io.VarLenFeature):
            inputs[key] = tf.keras.layers.Input(shape=[None], name=key, dtype=spec.dtype, sparse=True)
        elif isinstance(spec, tf.io.FixedLenFeature):
            inputs[key] = tf.keras.layers.Input(shape=spec.shape or [1], name=key, dtype=spec.dtype)
        else:
            raise ValueError('Spec type is not supported: ', key, spec)

    # Define the model architecture using the inputs
    output = tf.keras.layers.Concatenate()(tf.nest.flatten(inputs))
    output = tf.keras.layers.Dense(100, activation='relu')(output)
    output = tf.keras.layers.Dense(70, activation='relu')(output)
    output = tf.keras.layers.Dense(50, activation='relu')(output)
    output = tf.keras.layers.Dense(20, activation='relu')(output)
    output = tf.keras.layers.Dense(1)(output)
    return tf.keras.Model(inputs=inputs, outputs=output)

# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
    """Train the model based on given args.

    Args:
      fn_args: Holds args used to train the model as name/value pairs.
    """
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    # Create training and evaluation datasets using the input function
    train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor,
                              tf_transform_output, _BATCH_SIZE)
    eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor,
                             tf_transform_output, _BATCH_SIZE)

    # Build and compile the Keras model
    model = _build_keras_model(tf_transform_output)

    model.compile(
      optimizer=tf.optimizers.Adam(learning_rate=0.0005), 
      loss=tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.MeanSquaredError()])

    # Train the model using the training and evaluation datasets
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=fn_args.model_run_dir, update_freq='batch')

    model.fit(
        train_dataset,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        callbacks=[tensorboard_callback])

    # Export the trained model with serving signatures
    export_serving_model(tf_transform_output, model, fn_args.serving_model_dir)

Overwriting bfs_trainer.py


### Copy files to bucket
The transform and trainer module files need to be copied over to the GCP bucket for TFX to read.

In [17]:
!gsutil cp bfs_trainer.py {MODULE_ROOT}/
!gsutil cp transformer.py {MODULE_ROOT}/

Copying file://bfs_trainer.py [Content-Type=text/x-python]...
/ [1 files][  7.3 KiB/  7.3 KiB]                                                
Operation completed over 1 objects/7.3 KiB.                                      
Copying file://transformer.py [Content-Type=text/x-python]...
/ [1 files][  4.4 KiB/  4.4 KiB]                                                
Operation completed over 1 objects/4.4 KiB.                                      


### Create TFX pipeline. This pipeline can then be passed onto an orchestrator, such as KubeFlow, for deployment.

In [18]:

def build_pipeline(pipeline_name, pipeline_root, serving_model_dir, data_root, file_name):
    print("Running pipeline")

    print("Creating example_gen")
    example_gen = tfx.components.CsvExampleGen(input_base=data_root)

    print("Creating statistics_gen")
    statistics_gen = tfx.components.StatisticsGen(examples=example_gen.outputs["examples"])

    print("Creating schema_gen")
    schema_gen = tfx.components.SchemaGen(statistics=statistics_gen.outputs["statistics"])

    print("Creating example_validator")
    example_validator = tfx.components.ExampleValidator(
        statistics=statistics_gen.outputs["statistics"],
        schema=schema_gen.outputs["schema"]
    )

    print("Creating transform")
    transform = tfx.components.Transform(
        examples=example_gen.outputs["examples"],
        schema=schema_gen.outputs["schema"],
        module_file=os.path.join(MODULE_ROOT, "transformer.py")
    )

    print("Creating trainer")
    trainer = tfx.components.Trainer(
        examples=transform.outputs["transformed_examples"],
        transform_graph=transform.outputs["transform_graph"],
        module_file=os.path.join(MODULE_ROOT, "bfs_trainer.py"),
        schema=schema_gen.outputs["schema"],
        train_args=trainer_pb2.TrainArgs(num_steps=150),
        eval_args=trainer_pb2.EvalArgs(num_steps=150)
    )

    print("Adding model_resolver")
    model_resolver = tfx.dsl.Resolver(
        strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
        model=tfx.dsl.Channel(type=standard_artifacts.Model),
        model_blessing=tfx.dsl.Channel(type=standard_artifacts.ModelBlessing)
    ).with_id('latest_blessed_model_resolver')

    print("Adding evaluator")
    evaluator = tfx.components.Evaluator(
        examples=example_gen.outputs['examples'],
        model=trainer.outputs['model'],
        baseline_model=model_resolver.outputs['model'],
        eval_config=tfma.EvalConfig(
            model_specs=[
                tfma.ModelSpec(
                    signature_name='serving_default',
                    label_key='fare',
                    preprocessing_function_names=['transform_features'],
                )
            ],
            metrics_specs=[
                tfma.MetricsSpec(
                    metrics=[
                        tfma.MetricConfig(class_name='ExampleCount'),
                        tfma.MetricConfig(class_name='MeanSquaredError')
                    ]
                )
            ],
            slicing_specs=[
                tfma.SlicingSpec(feature_keys=['trip_start_hour'])
            ]
        )
    )

    print("Creating pusher")
    pusher = tfx.components.Pusher(
        model=trainer.outputs['model'],
        model_blessing=evaluator.outputs['blessing'],
        push_destination=pusher_pb2.PushDestination(
            filesystem=pusher_pb2.PushDestination.Filesystem(base_directory=serving_model_dir)
        )
    )

    print("Creating tfx_pipeline")
    tfx_pipeline = Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_root,
        components=[
            example_gen,
            statistics_gen,
            schema_gen,
            example_validator,
            transform,
            trainer,
            model_resolver,
            evaluator,
            pusher
        ],
        enable_cache=True
    )

    print("Creating runner")
    runner = tfx.orchestration.experimental.KubeflowV2DagRunner(
        config=tfx.orchestration.experimental.KubeflowV2DagRunnerConfig(),
        output_filename=pipeline_name + '_pipeline.json'
    )

    print("Executing runner")
    _ = runner.run(tfx_pipeline)

    print("Pipeline is ready to be executed.")
    
    
build_pipeline(PIPELINE_NAME, PIPELINE_ROOT, SERVING_MODEL_DIR, DATA_ROOT, FILE_NAME)

Running pipeline
Creating example_gen
Creating statistics_gen
Creating schema_gen
Creating example_validator
Creating transform
Creating trainer
Adding model_resolver
Adding evaluator
Creating pusher
Creating tfx_pipeline
Creating runner
Executing runner
running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying transformer.py -> build/lib
installing to /var/tmp/tmpbea9d4mu
running install


/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


running install_lib
copying build/lib/transformer.py -> /var/tmp/tmpbea9d4mu
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /var/tmp/tmpbea9d4mu/tfx_user_code_Transform-0.0+9a5a89b6638dd2d2f72fb9360d4a810643fc3eadbcdd9f0c43022d91098b93de-py3.10.egg-info
running install_scripts
creating /var/tmp/tmpbea9d4mu/tfx_user_code_Transform-0.0+9a5a89b6638dd2d2f72fb9360d4a810643fc3eadbcdd9f0c43022d91098b93de.dist-info/WHEEL
creating '/var/tmp/tmpc_xjpfbm/tfx_user_code_Transform-0.0+9a5a89b6638dd2d2f72fb9360d4a81

/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /var/tmp/tmp4i1spfdo/tfx_user_code_Trainer-0.0+0545d0a3143be72e8c4a7380209e47d36c8df9271e4bb0f0810855e6cd41d14a-py3.10.egg-info
running install_scripts
creating /var/tmp/tmp4i1spfdo/tfx_user_code_Trainer-0.0+0545d0a3143be72e8c4a7380209e47d36c8df9271e4bb0f0810855e6cd41d14a.dist-info/WHEEL
creating '/var/tmp/tmplkceb7o5/tfx_user_code_Trainer-0.0+0545d0a3143be72e8c4a7380209e47d36c8df9271e4bb0f0810855e6cd41d14a-py3-none-any.whl' and adding '/var/tmp/tmp4i1spfdo' to it
adding 'bfs_trainer.py'
adding 